In [4]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

# 设置请求头部信息，模拟浏览器请求
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36'}
# 设置YachtWorld网站上的搜索条件
search_params = {
    'class': 'sail',
    # 'price': 'USD100000-500000',
    # 'length': '40-50ft'
}
# base_url = 'https://www.yachtworld.com'

# requests.get(base_url, 
#     headers=headers)

In [5]:
# 使用 Concurrent.futures 模块中的 ThreadPoolExecutor 类来实现异步请求
import stable_get_html
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm
# pages = 1
pages = 353
# pages = 147
req_list1 = list(range(pages))
# sn = requests.Session()
def transform(i):
    # temp = requests.get(f'https://www.yachtworld.com/boats-for-sale/condition-used/type-sail/?page={i+1}', headers=headers).text
    # temp = stable_get_html.get_html(f'https://www.yachtworld.com/boats-for-sale/condition-used/type-sail/?page={i+1}').text
    temp = stable_get_html.get_html(f'https://www.yachtworld.com/boats-for-sale/condition-used/type-sail/region-europe/?page={i+1}').text
    # temp = stable_get_html.get_html(f'https://www.yachtworld.com/boats-for-sale/condition-new/type-sail/?page={i+1}').text
    req_list1[i] = BeautifulSoup(temp, 'html.parser')
with ThreadPoolExecutor(max_workers=64) as t:
    tasks = [t.submit(transform, i) for i in range(pages)]
    for future in tqdm(as_completed(tasks), total=len(tasks)):
        pass

100%|██████████| 353/353 [00:29<00:00, 12.12it/s]


In [6]:
soups = req_list1
all(map(lambda x: isinstance(x, BeautifulSoup), soups))

True

In [7]:
pages = 334
req_list2 = list(range(pages))
# sn = requests.Session()
def transform(i):
    temp = stable_get_html.get_html(f'https://www.yachtworld.com/boats-for-sale/condition-used/type-sail/region-northamerica/region-asia/region-oceania/region-africa/region-southamerica/?page={i+1}').text
    req_list2[i] = BeautifulSoup(temp, 'html.parser')
with ThreadPoolExecutor(max_workers=64) as t:
    tasks = [t.submit(transform, i) for i in range(pages)]
    for future in tqdm(as_completed(tasks), total=len(tasks)):
        pass

100%|██████████| 334/334 [00:28<00:00, 11.83it/s]


In [8]:
soups = req_list1+req_list2
all(map(lambda x: isinstance(x, BeautifulSoup), soups))

True

In [9]:
from typing import List
i = 0
def get_urls_from_search_page(soup:BeautifulSoup):
    global i
    i=i+1
    try:
        a_s = soup.select("#root > div.search > div.flex.flex-row > div.search-right-col > div.pagination-and-results-container.search-display > div.listings-container > a")
    except Exception as e:
        print(i)
        raise e
    return a_s
a_s = sum(map(get_urls_from_search_page, soups), [])
len(a_s)

10287

In [10]:
links = list(map(lambda x:x['href'] , a_s))

In [13]:
all(map(lambda x: x.startswith('https://'), links)) # 合法性
# [i for i in range(len(links)) if x.startswith('https://')]
len(links) # 数量
# unique的数量
len(set(links))


9987

In [16]:
def a2loaction(a):
    location = a.select_one(' div.listing-card-location')
    return location.text
    
locations = list(map(a2loaction, a_s))
all(locations)

True

In [17]:
def a2price(a):
    price = a.select_one(' div.price').text
    try:
        return float("".join(list(filter(str.isdigit, price))))
    except Exception as e:
        if price=='Call for Price': return None
        print(price)
        raise e
        
prices = list(map(a2price, a_s))
all(prices)
prices[:5]

[4892.0, 11765.0, 1902933.0, 260078.0, 49477.0]

In [18]:
df = pd.DataFrame({'Link':links, 'Price':prices, 'Basics.Location':locations})
df['New'] = False

In [19]:
df.head()
df.isna().sum()

Link                 0
Price              136
Basics.Location      0
New                  0
dtype: int64

In [20]:
df.to_csv('used_links.csv')

In [21]:
# 使用 Concurrent.futures 模块中的 ThreadPoolExecutor 类来实现异步请求
import stable_get_html
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm
# pages = 1
# pages = 666
pages = 147
req_list = list(range(pages))
# sn = requests.Session()
def transform(i):
    # temp = requests.get(f'https://www.yachtworld.com/boats-for-sale/condition-used/type-sail/?page={i+1}', headers=headers).text
    # temp = stable_get_html.get_html(f'https://www.yachtworld.com/boats-for-sale/condition-used/type-sail/?page={i+1}').text
    temp = stable_get_html.get_html(f'https://www.yachtworld.com/boats-for-sale/condition-new/type-sail/?page={i+1}').text
    req_list[i] = BeautifulSoup(temp, 'html.parser')
with ThreadPoolExecutor(max_workers=64) as t:
    tasks = [t.submit(transform, i) for i in range(pages)]
    for future in tqdm(as_completed(tasks), total=len(tasks)):
        pass


100%|██████████| 147/147 [00:32<00:00,  4.57it/s]


In [23]:
soups = req_list
all(map(lambda x: isinstance(x, BeautifulSoup), soups))
from typing import List
i = 0
def get_urls_from_search_page(soup:BeautifulSoup):
    global i
    i=i+1
    try:
        a_s = soup.select("#root > div.search > div.flex.flex-row > div.search-right-col > div.pagination-and-results-container.search-display > div.listings-container > a")
    except Exception as e:
        print(i)
        raise e
    return a_s
a_s = sum(map(get_urls_from_search_page, soups), [])
len(a_s)
links = list(map(lambda x:x['href'] , a_s))

def a2loaction(a):
    location = a.select_one(' div.listing-card-location')
    return location.text
    
locations = list(map(a2loaction, a_s))
all(locations)

def a2price(a):
    
    price = a.select_one(' div.price')
    if price is None: 
        print(a)
        return None
    price = price.text
    try:
        
        return float("".join(list(filter(str.isdigit, price))))
    except Exception as e:
        if price=='Call for Price': return None
        print(price)
        raise e
        
prices = list(map(a2price, a_s))
all(prices)
prices[:5]

<a data-reporting-click-listing-type="standard listing" data-reporting-click-product-id="8395584" data-reporting-page="64" data-reporting-rank="953" href="https://www.yachtworld.com/yacht/2023-lagoon-42-8395584/" name="&amp;lpos=P1S1&amp;lid=SR_Detail_Photo" property="itemListElement" typeof="ListItem"><meta content="https://www.yachtworld.com/yacht/2023-lagoon-42-8395584/" property="url"/><meta content="8" property="position"/><div class="row listing-card regular" id="8395584"><div class="listing-card-image-container"><meta content="//images.yachtworld.com/resize/1/88/67/6808867_20180815040902966_1_LARGE.jpg?w=243&amp;h=155&amp;t=1657724328" property="image"/><span class="listing-card-image" style="display:inline-block;width:243px;height:155px"></span></div><div class="listing-card-information"><h2 class="listing-card-title blue-star-title" property="name">Lagoon 42</h2><div class="listing-card-length-year">42 ft<!-- --> / <!-- -->2023</div><div class="listing-card-location"></div><p 

[329795.0, 364999.0, 432319.0, 31000.0, 850000.0]

In [24]:
df = pd.DataFrame({'Link':links, 'Price':prices, 'Basics.Location':locations})
df['New'] = True
df.head()

,Link,Price,Basics.Location,New
0,https://www.yachtworld.com/yacht/2023-jeanneau...,329795.0,"Seattle, Washington, United States",True
1,https://www.yachtworld.com/yacht/2023-dufour-3...,364999.0,"Racine, Wisconsin, United States",True
2,https://www.yachtworld.com/yacht/2023-bavaria-...,432319.0,"San Diego, California, United States",True
3,https://www.yachtworld.com/yacht/1984-union-po...,31000.0,"Emeryville, California, United States",True
4,https://www.yachtworld.com/yacht/2023-dufour-4...,850000.0,"Racine, Wisconsin, United States",True


In [25]:
df.isna().sum()


Link                  0
Price              1031
Basics.Location       0
New                   0
dtype: int64

In [30]:
len(df.Link.unique())

2188

In [31]:
df['Basics.Location'].isna().to_numpy()

array([False, False, False, ..., False, False, False])

In [32]:
import random, numpy as np
# indexes = random.choices(list(range(len(df))), k=10)
indexes = random.choices(np.argwhere(df['Price'].isna().to_numpy()), k=10)
prices = np.array(prices)
links = np.array(links)
prices[indexes], links[indexes]

(array([[None],
        [None],
        [None],
        [None],
        [None],
        [None],
        [None],
        [None],
        [None],
        [None]], dtype=object),
 array([['https://www.yachtworld.com/yacht/2023-moody-aft-cockpit-41-8115025/'],
        ['https://www.yachtworld.com/yacht/2023-x--yachts-xp-50-4530040/'],
        ['https://www.yachtworld.com/yacht/2023-beneteau-oceanis-40.1-7892889/'],
        ['https://www.yachtworld.com/yacht/2023-tartan-455-8654222/'],
        ['https://www.yachtworld.com/yacht/2023-bavaria-c46----new-model-8649239/'],
        ['https://www.yachtworld.com/yacht/2022-corsair-760-sport-7624491/'],
        ['https://www.yachtworld.com/yacht/2023-moody-decksaloon-54-8115027/'],
        ['https://www.yachtworld.com/yacht/2023-catalina-275-4442194/'],
        ['https://www.yachtworld.com/yacht/2024-grand-soleil-80-custom-7424626/'],
        ['https://www.yachtworld.com/yacht/2023-jeanneau-65-8273829/']],
       dtype='<U102'))

In [33]:
df.to_csv('new_links.csv')

In [34]:
len(df)

2188

In [35]:
df_used = pd.read_csv('used_links.csv', index_col=0)
df_used.head()

,Link,Price,Basics.Location,New
0,https://www.yachtworld.com/yacht/1965-folkdanc...,4892.0,"Suffolk, United Kingdom",False
1,https://www.yachtworld.com/yacht/1971-she-31-8...,11765.0,"Falmouth UK, United Kingdom",False
2,https://www.yachtworld.com/yacht/1991-jongert-...,1902933.0,"Mallorca, Spain",False
3,https://www.yachtworld.com/yacht/1972-nautor-s...,260078.0,"Lagos, Portugal",False
4,https://www.yachtworld.com/yacht/1990-beneteau...,49477.0,"Brighton Marina Village, East Sussex, United K...",False


In [36]:
len(df_used)

10287

In [37]:
df_all = pd.concat([df, df_used])
df_all.shape

(12475, 4)

In [ ]:
df_all.to_csv('all_links.csv')